In [48]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
from Wick import *

In [49]:
norbs = 3
Nocc = 2
vacuumOccupations = [1 for i in range(Nocc)] + [0 for i in range (norbs - Nocc)]
print(vacuumOccupations)

[1, 1, 0]


In [50]:
oP = operatorProduct([basicOperator(2,0,1), basicOperator(1,0,1), basicOperator(2,1,1), basicOperator(1,1,1)])
nOP = normalOrder(oP, vacuumOccupations)
print(oP)
print(nOP)
print(wickExpand(oP, vacuumOccupations))
print(vacuumExpectationValue(oP, vacuumOccupations))

1.0 * a_{2\alpha}a_{1\alpha}a^{2\alpha}a^{1\alpha}
1.0 * a_{1\alpha}a^{2\alpha}a_{2\alpha}a^{1\alpha}
1.0 * a_{1\alpha}a^{2\alpha}a_{2\alpha}a^{1\alpha} + -1.0 * a_{1\alpha}a^{1\alpha}
0.0


In [51]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [52]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [53]:
eri

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [54]:
mf.energy_elec() + mf.energy_nuc()

array([-1.10263881,  1.27706243])

In [55]:
mf.mo_coeff

array([[ 0.56092571, -1.10314488],
       [ 0.56092571,  1.10314488]])

In [56]:
mf.nelec

(1, 1)

In [57]:
def spinFreeSingleExcitation(p, q):
    summandList = []
    for spin in range(2):
        summandList.append(operatorProduct([basicOperator(p,1,spin), basicOperator(q,0,spin)]))
    return operatorSum(summandList)

In [58]:
def spinFreeDoubleExcitation(p, q, r, s):
    summandList = []
    for spin1 in range(2):
        for spin2 in range(2):
            summandList.append(operatorProduct([basicOperator(p,1,spin2), basicOperator(q,1,spin1), basicOperator(s,0,spin1), basicOperator(r,0,spin2)]))
    return operatorSum(summandList)

In [59]:
Norbs = mol.nao

In [60]:
hamiltonian1Body = operatorSum([])
for p in range(Norbs):
    for q in range(Norbs):
        hamiltonian1Body = hamiltonian1Body + h1[p, q] * spinFreeSingleExcitation(p, q)

In [61]:
print(hamiltonian1Body)

-1.1898506186070186 * a^{0\beta}a_{0\beta} + -1.1898506186070186 * a^{0\alpha}a_{0\alpha} + 2.600212552842601e-17 * a^{0\beta}a_{1\beta} + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha} + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta} + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha} + -0.5337491016607854 * a^{1\beta}a_{1\beta} + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}


In [62]:
print(wickExpand(hamiltonian1Body, [1,0]))

1.1898506186070186 * a_{0\beta}a^{0\beta} + -1.1898506186070186 + 1.1898506186070186 * a_{0\alpha}a^{0\alpha} + -1.1898506186070186 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta} + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha} + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta} + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha} + -0.5337491016607854 * a^{1\beta}a_{1\beta} + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}


In [63]:
hamiltonian1Body

In [64]:
wickExpand(hamiltonian1Body, [1,0])

In [65]:
for s in hamiltonian1Body.summandList:
    print(s)

-1.1898506186070186 * a^{0\beta}a_{0\beta}
-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}


In [66]:
for s in normalOrder(hamiltonian1Body, [1,0]).summandList:
    print(s)

1.1898506186070186 * a_{0\beta}a^{0\beta}
1.1898506186070186 * a_{0\alpha}a^{0\alpha}
2.600212552842601e-17 * a^{0\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}


In [67]:
for s in wickExpand(hamiltonian1Body, [1,0]).summandList:
    print(s)

1.1898506186070186 * a_{0\beta}a^{0\beta}
-1.1898506186070186
1.1898506186070186 * a_{0\alpha}a^{0\alpha}
-1.1898506186070186
2.600212552842601e-17 * a^{0\beta}a_{1\beta}
2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
-4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
-4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
-0.5337491016607854 * a^{1\beta}a_{1\beta}
-0.5337491016607854 * a^{1\alpha}a_{1\alpha}


In [68]:
w = wickExpand(hamiltonian1Body, [1,0])

In [69]:
w

In [70]:
print(w)

1.1898506186070186 * a_{0\beta}a^{0\beta} + -1.1898506186070186 + 1.1898506186070186 * a_{0\alpha}a^{0\alpha} + -1.1898506186070186 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta} + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha} + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta} + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha} + -0.5337491016607854 * a^{1\beta}a_{1\beta} + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}


In [71]:
vacuumExpectationValue(hamiltonian1Body, [1,0])

-2.379701237214037

In [72]:
vEV = 0.
for summand in w.summandList:
    if summand.operatorList == []:
        print(summand)
        vEV += summand.prefactor


-1.1898506186070186
-1.1898506186070186


In [73]:
vEV

-2.379701237214037

In [74]:
mf.energy_nuc()

0.6230529146661897

In [75]:
mf.energy_elec()

(-1.7256917258579134, 0.6540095113561237)

In [76]:
hamiltonian2Body = operatorSum([])
for p in range(Norbs):
    for q in range(p, Norbs):
        for r in range(Norbs):
            for s in range(r, Norbs):
                x = int(q + Norbs * p - 0.5 * p * (p + 1))
                y = int(s + Norbs * r - 0.5 * r * (r + 1))
                hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)

In [77]:
for s in wickExpand(hamiltonian2Body, [1,0]).summandList:
    print(s)

0.3270047556780619 * a_{0\beta}a_{0\beta}a^{0\beta}a^{0\beta}
0.3270047556780619 * a_{0\beta}a^{0\beta}
-0.3270047556780619 * a_{0\beta}a^{0\beta}
-0.3270047556780619 * a_{0\beta}a^{0\beta}
0.3270047556780619 * a_{0\beta}a^{0\beta}
-0.3270047556780619
0.3270047556780619
0.3270047556780619 * a_{0\beta}a_{0\alpha}a^{0\alpha}a^{0\beta}
-0.3270047556780619 * a_{0\beta}a^{0\beta}
-0.3270047556780619 * a_{0\alpha}a^{0\alpha}
0.3270047556780619
0.3270047556780619 * a_{0\alpha}a_{0\beta}a^{0\beta}a^{0\alpha}
-0.3270047556780619 * a_{0\alpha}a^{0\alpha}
-0.3270047556780619 * a_{0\beta}a^{0\beta}
0.3270047556780619
0.3270047556780619 * a_{0\alpha}a_{0\alpha}a^{0\alpha}a^{0\alpha}
0.3270047556780619 * a_{0\alpha}a^{0\alpha}
-0.3270047556780619 * a_{0\alpha}a^{0\alpha}
-0.3270047556780619 * a_{0\alpha}a^{0\alpha}
0.3270047556780619 * a_{0\alpha}a^{0\alpha}
-0.3270047556780619
0.3270047556780619
-4.163336342344337e-17 * a_{0\beta}a^{0\beta}a^{0\beta}a_{1\beta}
4.163336342344337e-17 * a^{0\beta}a_{1

In [78]:
vacuumExpectationValue(hamiltonian2Body, [1,0])

0.6540095113561238

In [79]:
vacuumExpectationValue(hamiltonian1Body, [1,0]) + vacuumExpectationValue(hamiltonian2Body, [1,0])

-1.7256917258579132

In [80]:
vacuumExpectationValue(hamiltonian2Body, [1,0])

0.6540095113561238

In [81]:
mf.energy_elec()

(-1.7256917258579134, 0.6540095113561237)

In [82]:
hamiltonian = hamiltonian1Body + hamiltonian2Body

In [83]:
vacuumExpectationValue(hamiltonian, [1,0])

-1.7256917258579132

In [ ]:
amplitudes = []

In [ ]:
for 